In [1]:
import pandas as pd
from sqlalchemy import create_engine

Let's define the connection string, create our engine, and build our query

In [2]:
op10 = (
    'mssql+pyodbc:///?odbc_connect='
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=srvvmop1022;'
    'DATABASE=COGWASTEWATER;'
    'Trusted_Connection=yes;'
)

op10_engine = create_engine(op10)

op10_query = """
    SELECT LocName, ParName, DateTime, Value, 'bc' AS plant FROM dbo.DATA_006_data
    UNION ALL
    SELECT LocName, ParName, DateTime, Value, 'ch' AS plant FROM dbo.DATA_007_data
    UNION ALL
    SELECT LocName, ParName, DateTime, Value, 'pb' AS plant FROM dbo.DATA_008_data
    UNION ALL
    SELECT LocName, ParName, DateTime, Value, 'ds' AS plant FROM dbo.DATA_009_data
    UNION ALL
    SELECT LocName, ParName, DateTime, Value, 'sg' AS plant FROM dbo.DATA_010_data
"""

Let's read the query results into a dataframe using `read_sql`

In [3]:
df = pd.read_sql(op10_query, op10_engine)

Here we can filter the dataframe to meet our needs:

In [4]:
df_p = df[(df.LocName == 'Plant discharge') & (df.plant == 'sg') & (df.DateTime >= '202409010000')].reset_index(drop=True)
df_p.drop(columns=['LocName', 'plant'], inplace=True)
df_p.DateTime = pd.to_datetime(df_p.DateTime, format='%Y%m%d%H%M')
df_p.head()

,ParName,DateTime,Value
0,BOD 5,2024-09-03,2.0
1,BOD Carb 5,2024-09-03,2.0
2,Chloride,2024-09-03,198.0
3,Nh3 N Ammonia,2024-09-03,0.3
4,Solids Total,2024-09-03,706.0


Let's now pivot the dataframe to match the format of the Operator10 dashboard. 

In [5]:
df_p = df_p.pivot(index='DateTime', columns='ParName', values='Value')
df_p.head()

ParName,2 hr peak Mgd,2 hr peak gpm,365 day avg flow,Annual Avg Flo,BOD 5,BOD 7d average,BOD Carb 5,CBOD 7d average,Chloride,Chloride lbs,...,Sulfate,TSS 7d average,Temp Water C,avg lbs of P,lbs of BOD,lbs of CBOD,lbs of NH3,lbs of TSS,lbs of total P,pH
DateTime,,,,,,,,,,,,,,,,,,,,,
2024-09-01,2.888,2006.0,1.821497,1.817524,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-02,3.281,2278.0,1.823300,1.817524,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-03,3.114,2163.0,1.825607,1.817524,2.0,NaN,2.0,NaN,198.0,3462.81804,...,51.7,NaN,28.1,NaN,34.97796,34.97796,5.246694,34.97796,17.48898,7.62
2024-09-04,5.074,3524.0,1.827541,1.817524,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,28.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-09-05,3.484,2419.0,1.831892,1.817524,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_p.to_csv('test.csv')